# Import

In [ ]:
!pip install mlflow
!pip install torch==1.8.1
!pip install pytorch-lightning
!pip install transformers
!pip install cloudpathlib

In [1]:
import sys
sys.path.append('../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [2]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import mlflow
import mlflow.sagemaker
import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    T5ForConditionalGeneration,
    T5Tokenizer,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

In [3]:
from deep.constants import *
from deep.utils import *

In [4]:
%load_ext autoreload
%autoreload 2

## Data

In [15]:
logged_model = 's3://deep-mlflow-artifact/2/f3b4e0f9a0364f8dbfe1563b248348a1/artifacts/model'

## Sagemaker Prep

In [16]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
prefix = "pl/example"

In [17]:
mlflow.sagemaker.deploy(
    'pl-example',
    logged_model,
    execution_role_arn=SAGEMAKER_ROLE_ARN,
    image_url='961104659532.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:latest',
    region_name='us-east-1',
    instance_type='ml.p2.xlarge',
    mode='replace'
)

2021/07/12 20:16:33 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2021/07/12 20:16:34 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2021/07/12 20:16:35 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-1-961104659532` already exists. Skipping creation.
2021/07/12 20:18:38 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': 'VST2QV9A6EN08VX1', 'HostId': 'b/sF9HIJUDKfbbqcP30mAFDFIX0ZtOaOo/t4002V8TXxQgy6kNuUf0csuMDkR+fW4LI04c4M1gI=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'b/sF9HIJUDKfbbqcP30mAFDFIX0ZtOaOo/t4002V8TXxQgy6kNuUf0csuMDkR+fW4LI04c4M1gI=', 'x-amz-request-id': 'VST2QV9A6EN08VX1', 'date': 'Mon, 12 Jul 2021 18:18:39 GMT', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}}
2021/07/12 20:18:40 INFO mlflow.sagemaker: Found active endpoint with arn: arn:aws:sagemaker:us-east-1:961104659532:endpoint/pl-example. Updating...
2021/07/12 20:18:40 INFO mlflow.sagemaker